In [58]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup

In [59]:
Toronto_postal_code_link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# request wikipage 
Toronto_raw_page = requests.get(Toronto_postal_code_link)
bs=BeautifulSoup(Toronto_raw_page.text,"lxml")
table_body=bs.find('tbody')

# tabel part
rows = table_body.find_all('tr')
heads = table_body.find_all('th')

In [60]:
heads=list(heads)

In [61]:
def clean(text):
    '''clean table header for column names'''
    text=text.replace('<th>','').replace("</th>",'').replace("\n",'')
    return text

col_name = [clean(str(w)) for w in heads]
col_name

['Postcode', 'Borough', 'Neighbourhood']

In [62]:
df=pd.DataFrame(columns=col_name)
for i,row in enumerate(rows[1:]):
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    df.loc[i] = cols

In [63]:
df.shape

(287, 3)

In [64]:
#df.head(10)

In [65]:
sorted(df['Borough'].unique())

['Central Toronto',
 'Downtown Toronto',
 'East Toronto',
 'East York',
 'Etobicoke',
 'Mississauga',
 'North York',
 'Not assigned',
 'Scarborough',
 'West Toronto',
 'York']

In [66]:
#df1=df[~df['Borough'].isin(['Not assigned',''])]

In [67]:
#df1.shape

In [68]:
#df1.head(10)

In [69]:
#df2=df1.groupby(['Postcode','Borough'])['Neighbourhood'].agg(lambda x: ','.join(x)).reset_index()

In [70]:
#df2.loc[df2['Neighbourhood']=='Not assigned',['Neighbourhood']]=df2['Borough']

In [71]:
#df2.shape

Part2. Get the latitude and the longitude coordinates of each neighborhood
link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

In [72]:
csv_zip = pd.read_csv('http://cocl.us/Geospatial_data')

In [73]:
csv_zip.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [74]:
csv_zip.rename(columns={'Postal Code' : 'Postcode'},inplace=True)
csv_zip.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [75]:
df2_zip=df2.merge(csv_zip, on=['Postcode'],how="left")

In [76]:
df2_zip.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [77]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Folium installed
Libraries imported.


In [78]:
CLIENT_ID = 'VBSBYNY2VBFL0HR4KT0ACIIXSHLZDSFT3HIRAWSW5WO4OXBW' # your Foursquare ID
CLIENT_SECRET = 'UGRKTBG241NH0K2ERHHCY3S1I3T3W1YOANUYBQW5ANU13WQH' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [82]:
df_Toronto = df2_zip[df2_zip['Borough'].str.contains('Toronto')]
#df_Toronto

In [83]:
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode('Toronto, ON')
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


# Q3. Explore and cluster the neighborhoods in Toronto.
add enough Markdown cells to explain what you decided to do and to report any observations you make.
generate maps to visualize your neighborhoods and how they cluster together.

In [85]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to the map
markers_colors = []

# add markers to map
for lat, lng, Borough, Postcode in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'],df_Toronto['Postcode']):
    label = Borough + ' (' + Postcode + ')'
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto